In [3]:
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright
from renetti.ws.spiders.utils import parse_product_json_ld_from_page
import aiohttp

In [4]:
url = "https://www.roguefitness.com/gb/weightlifting-bars-plates/barbells"
base_url = "https://www.roguefitness.com"

In [47]:
async with async_playwright() as playwright:
    async with await playwright.chromium.launch(headless=False) as browser:
        async with await browser.new_context() as context:
            async with await context.new_page() as page:
                urls = []
                page_number = 1
                while True:
                    await page.goto(
                        url=f"{url}?page_number={page_number}"
                    )
                    html = await page.content()
                    soup = BeautifulSoup(markup=html, features="html.parser")
                    a_tags = soup.findAll("a", class_="hover-card")
                    urls += [
                        f"{base_url}{a.get('href')}" for a in a_tags
                    ]
                    page_number += 1
                    try:
                        await page.wait_for_selector('a[aria-label="Category Pagination Next"]', timeout=2000)
                    except:
                        break

In [11]:
url = "https://www.roguefitness.com/gb/rogue-28mm-training-bar-cerakote"

In [13]:
async with aiohttp.ClientSession() as session:
        async with session.get(url=url) as response:
            html = await response.text()
            soup = BeautifulSoup(markup=html)
        product = parse_product_json_ld_from_page(soup=soup)
product

{'name': 'Rogue 28mm Training Bar - Cerakote',
 'image_links': ['https://assets.roguefitness.com/f_auto,q_auto,c_limit,w_1600,b_rgb:ffffff/catalog/Weightlifting%20Bars%20and%20Plates/Barbells/Mens%2020KG%20Barbells/28TRAININGCERAKOTE/28TRAININGCERAKOTE-BLUE-web2_runy8i.png',
  'https://assets.roguefitness.com/f_auto,q_auto,c_limit,w_1600,b_rgb:ffffff/catalog/Weightlifting%20Bars%20and%20Plates/Barbells/Mens%2020KG%20Barbells/28TRAININGCERAKOTE/28TRAININGCERAKOTE-BLACK-H_lek9li.png',
  'https://assets.roguefitness.com/f_auto,q_auto,c_limit,w_1600,b_rgb:ffffff/catalog/Weightlifting%20Bars%20and%20Plates/Barbells/Mens%2020KG%20Barbells/28TRAININGCERAKOTE/28TRAININGCERAKOTE-BLUE-web1_hoy4co.png',
  'https://assets.roguefitness.com/f_auto,q_auto,c_limit,w_1600,b_rgb:ffffff/catalog/Weightlifting%20Bars%20and%20Plates/Barbells/Mens%2020KG%20Barbells/28TRAININGCERAKOTE/28TRAININGCERAKOTE-BLUE-web4_xinfde.png',
  'https://assets.roguefitness.com/f_auto,q_auto,c_limit,w_1600,b_rgb:ffffff/catalog